<a href="https://colab.research.google.com/github/Alirs2002/tweet_sentiment_analysis/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this motherfucker project, i am going to create a model by keras,to classify tweets,to 2 classes

1-disaster

2-normal

lets import necessary libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

now,lets download our dataset

In [2]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-10-05 14:21:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.207, 172.253.115.207, 172.253.122.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-10-05 14:21:06 (156 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
!ls

nlp_getting_started.zip  sample_data


In [4]:
!ls ..

bin			    etc     media		      root  tools
boot			    home    mnt			      run   usr
content			    lib     NGC-DL-CONTAINER-LICENSE  sbin  var
cuda-keyring_1.0-1_all.deb  lib32   opt			      srv
datalab			    lib64   proc		      sys
dev			    libx32  python-apt		      tmp


now,lets create a function to unzip our data

In [6]:
import zipfile

def unzip_data(path):

  zip_file_ref = zipfile.ZipFile(path,"r")

  zip_file_ref.extractall()

  zip_file_ref.close()

In [7]:
zip_file_path = "./nlp_getting_started.zip"

In [8]:
unzip_data(zip_file_path)

In [9]:
!ls

nlp_getting_started.zip  sample_submission.csv	train.csv
sample_data		 test.csv


now,lets import our train and test datasets

In [10]:
train_data_path = "./train.csv"

test_data_path = "./test.csv"

In [11]:
train_data = pd.read_csv(train_data_path)

test_data = pd.read_csv(test_data_path)

In [12]:
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [27]:
pd.options.display.max_colwidth=200

In [28]:
train_data[:2][["text","target"]]

,text,target
0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,Forest fire near La Ronge Sask. Canada,1


In [29]:
train_data["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [31]:
train_data.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d,1


In [32]:
print(len(train_data))

7613


In [34]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [41]:
pd.options.display.max_dir_items=200

In [43]:
train_data["keyword"].isnull().value_counts()

False    7552
True       61
Name: keyword, dtype: int64

In [49]:
train_data[100:105]

,id,keyword,location,text,target
100,144,accident,UK,.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad,1
101,145,accident,"Nairobi, Kenya",I still have not heard Church Leaders of Kenya coming forward to comment on the accident issue and disciplinary measures#ArrestPastorNganga,0
102,146,aftershock,Instagram - @heyimginog,@afterShock_DeLo scuf ps live and the game... cya,0
103,149,aftershock,304,'The man who can drive himself further once the effort gets painful is the man who will win.' \nRoger Bannister,0
104,151,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/yNXnvVKCDA | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #IcesÛ_ http://t.co/weQPesENku,0


i think ,location and keyword,doesnt have any influenct on our prediction,so we will remove them from our dataset in future

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
train_data = train_data.sample(frac=1,random_state=85)

In [52]:
train_data

,id,keyword,location,text,target
6039,8633,seismic,"Mogadishu, Somalia",Exploration Takes Seismic Shift in #Gabon to #Somalia\nhttp://t.co/Ltf6jL5keU http://t.co/Zlq8tHcTkW,1
1709,2465,collided,San Francisco,Mind blown by @GlassAnimals slithering viscous Gold Mine (cover of Yeah Yeah Yeahs + Erykah Badu)Û_ http://t.co/7Zb9gm5z0h,0
2775,3992,devastation,NaN,70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/vFCtrzaOk2,1
2511,3607,desolation,"North East, England",Just came back from camping and returned with a new song which gets recorded tomorrow. Can't wait! #Desolation #TheConspiracyTheory #NewEP,1
5234,7480,obliteration,NaN,Dead Space - Obliteration Imminent [2/2]: http://t.co/XJB0dCAaHf via @YouTube,0
...,...,...,...,...,...
7053,10106,typhoon,Dhaka,Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/lEYJwNnAH8,1
1587,2292,cliff%20fall,"Lagos, Nigeria",Huge cliff landslide on road in China: Watch the moment a cliff collapses as huge chunks of rock fall onto a r... http://t.co/eEEwO207mX,1
5828,8321,rubble,Spain - China - Latin America.,China's Stock Market Crash: Are There Gems In The Rubble?: ChinaÛªs stock market crash this summer h... http://t.co/pE2R3lN16o by .Forbes,0
5280,7545,outbreak,Chile,Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/02ELqLOpFk,1


In [60]:
train_sentence,val_sentence,train_labels,val_labels = train_test_split(train_data["text"].to_numpy(),
                                                                       train_data["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=85)

In [61]:
len(train_sentence),len(train_labels),len(val_sentence),len(val_labels)

(6851, 6851, 762, 762)

In [64]:
train_sentence

array(['Insane bush fires in California. Be safe. https://t.co/jSlxTQ3NqS',
       '@beckyfeigin I defs will when it stops bleeding!',
       "I don't laugh out loud at many things. But man I really lol @ the big bang theory.",
       ...,
       'Only one man Tsutomi Yamaguchi is said to have survived both atomic bomb blasts at #Hiroshima and #Nagasaki. #OTD http://t.co/DaalPeNZp0',
       'Blue Bell May Be Close to a Return From Its Listeria Disaster... Hot on #theneeds #Recipes http://t.co/F56v61AmPt',
       '9:35 pm. Thunderstorm. No rain. 90 degrees. This weather weird.'],
      dtype=object)

now,we have created our training sentence

In [71]:
import random
def show_random(dataset1,dataset2):

  if(len(dataset1)!=len(dataset2)):
    return "fuck you,your fucking datasets are wrong"

  length = len(dataset1)

  random_num = random.randint(0,length)

  random_text = dataset1[random_num]
  random_label = dataset2[random_num]

  if(random_label ==1):
    random_label = "disaster"
  else:
    random_label = "normal"

  print(f"text: {random_text} \n label :{random_label}")

In [78]:
show_random(train_sentence,train_labels)

text: I had to grill for a school function. One of the grills we had going was pretty much either off or forest fire. No inbetween! Made it work 
 label :normal


ok,now we should do our next step which is converting our dataset to number